In [1]:
!nvidia-smi 

Sun Jan 31 15:46:39 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# @title Download dataset
import os
import shutil
import time
from google_drive_downloader import GoogleDriveDownloader as gdd

if not os.path.exists("/content/images/valid.zip"):
    # https://drive.google.com/file/d/1ux-mv7250XsgZo-vsk4QIedfGbELl9oZ/view?usp=sharing
    # LR - 64x64 images
    gdd.download_file_from_google_drive(file_id='1ux-mv7250XsgZo-vsk4QIedfGbELl9oZ',
                                        dest_path='./images/valid.zip',
                                        unzip=True,
                                        showsize=True,
                                        )
    # !rm -rf /content/images/valid.zip

# download scripts
!rm -rf *.py
time.sleep(2)
!wget -q https://raw.githubusercontent.com/veb-101/Esrgan-pytorch/master/trainer.py -O trainer.py
!wget -q https://raw.githubusercontent.com/veb-101/Esrgan-pytorch/master/utils.py -O utils.py
!wget -q https://raw.githubusercontent.com/veb-101/Esrgan-pytorch/master/models.py -O models.py

# https://drive.google.com/file/d/1ak35jClzM1kAf7p4325b5P9CIudfuwWq/view?usp=sharing

if not os.path.exists("/content/pretrained.zip"):
    # https://drive.google.com/file/d/1ak35jClzM1kAf7p4325b5P9CIudfuwWq/view?usp=sharing
    # HR - 256x256 images
    gdd.download_file_from_google_drive(file_id='1ak35jClzM1kAf7p4325b5P9CIudfuwWq',
                                        dest_path='./pretrained.zip',
                                        unzip=True,
                                        showsize=True,
                                        )


if not os.path.exists("/content/images/train.zip"):
    # https://drive.google.com/file/d/1RGvBO7wVCI4aPNkjy8w-Sl3Kzp2UCqXm/view?usp=sharing
    # HR - 256x256 images
    gdd.download_file_from_google_drive(file_id='1RGvBO7wVCI4aPNkjy8w-Sl3Kzp2UCqXm',
                                        dest_path='./images/train.zip',
                                        unzip=True,
                                        showsize=True,
                                        )
    # !rm -rf /content/images/train.zip

7.9 MiB Done.
Unzipping...Done.
119.8 MiB Done.
Unzipping...Done.
1.4 GiB Done.


In [11]:
import trainer
import os
import random
from PIL import Image
import numpy as np

import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms.functional as TF
import importlib

import warnings
warnings.simplefilter("ignore", UserWarning)

seed = 41
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)


def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device("cuda")
    else:
        return torch.device("cpu")


device = get_default_device()


class ESR_Dataset(Dataset):
    def __init__(self, num_images=9000, path=r"images", train=True):
        self.path = path
        self.is_train = train

        if not os.path.exists(self.path):
            raise Exception(f"[!] dataset is not exited")

        self.image_paths = os.listdir(os.path.join(self.path, "hr"))
        self.image_file_name = np.random.choice(
            self.image_paths, size=num_images, replace=False)

        self.mean = np.array([0.485, 0.456, 0.406])
        self.std = np.array([0.229, 0.224, 0.225])
        
    def __getitem__(self, item):
        file_name = self.image_file_name[item]
        high_resolution = Image.open(os.path.join(self.path, "hr", file_name)).convert(
            "RGB"
        )
        low_resolution = Image.open(os.path.join(self.path, "lr", file_name)).convert(
            "RGB"
        )

        if self.is_train:
            if random.random() > 0.5:
                high_resolution = TF.vflip(high_resolution)
                low_resolution = TF.vflip(low_resolution)

            if random.random() > 0.5:
                high_resolution = TF.hflip(high_resolution)
                low_resolution = TF.hflip(low_resolution)

            if random.random() > 0.5:
                high_resolution = TF.rotate(high_resolution, 90)
                low_resolution = TF.rotate(low_resolution, 90)

        high_resolution = TF.to_tensor(high_resolution)
        low_resolution = TF.to_tensor(low_resolution)

        # high_resolution = TF.normalize(high_resolution, self.mean, self.std)
        # low_resolution = TF.normalize(low_resolution, self.mean, self.std)

        images = {"lr": low_resolution, "hr": high_resolution}

        return images

    def __len__(self):
        return len(self.image_file_name)



def make_dataloaders(
    batch_size=32, n_workers=4, shuffle=True, **kwargs):  # A handy function to make our dataloaders
    dataset = ESR_Dataset(**kwargs)

    pin = torch.cuda.is_available()

    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        num_workers=n_workers,
        pin_memory=pin,
        shuffle=shuffle,
    )
    return dataloader


In [12]:
config = {
    "image_size": 256,
    "batch_size": 16,
    "start_epoch": 0,
    "num_epoch": 100,
    "sample_batch_size": 1,
    "checkpoint_dir": "./checkpoints",
    "sample_dir": "./samples",
    "workers": 6,
    "scale_factor": 4,
    "num_rrdn_blocks": 23,
    "nf": 64,
    "gc": 32,
    "b1": 0.9,
    "b2": 0.999,
    "weight_decay": 1e-2,
    # ------ PSNR ------
    "p_lr": 2e-4,
    "p_decay_iter": [30, 60, 90],
    "p_perceptual_loss_factor": 0,
    "p_adversarial_loss_factor": 0,
    "p_content_loss_factor": 1,
    # ------------------
    # ------ ADVR ------
    "g_lr": 1e-4,
    "g_decay_iter": [25, 50, 80],
    "g_perceptual_loss_factor": 1,
    "g_adversarial_loss_factor": 5e-3,
    "g_content_loss_factor": 1e-2,
    # ------------------
    "is_psnr_oriented": True,
    "load_previous_opt": True,
}

In [13]:
import trainer
import models
import utils

importlib.reload(utils)
importlib.reload(models)
importlib.reload(trainer)

if not os.path.exists(config["sample_dir"]):
    os.makedirs(config["sample_dir"])

In [7]:
esr_dataloader_train = make_dataloaders(batch_size=config["batch_size"], 
                                        n_workers=config["workers"], 
                                        shuffle=True, 
                                        num_images=9000, 
                                        path=r"./images/train", 
                                        train=True)

esr_dataloader_val = make_dataloaders(batch_size=config["batch_size"],
                                      n_workers=config["workers"], 
                                      shuffle=False, 
                                      num_images=64, 
                                      path=r"./images/valid", 
                                      train=False)

In [ ]:
## @title warmup training
pin = torch.cuda.is_available()

config["start_epoch"] = 0
config["num_epoch"] = 120
config["is_psnr_oriented"] = True
config["load_previous_opt"] = False
# config["load_previous_opt"] = True # uncomment for subsequent runs of same type


for key, value in config.items():
    print(f"{key:30}: {value}")

print("\n\n")
print(f"ESRGAN start")

torch.cuda.empty_cache()
psnr_model = trainer.Trainer(
    config, esr_dataloader_train, esr_dataloader_val, device)
psnr_model_metrics = psnr_model.train()

image_size                    : 256
batch_size                    : 16
start_epoch                   : 0
num_epoch                     : 120
sample_batch_size             : 1
checkpoint_dir                : ./checkpoints
sample_dir                    : ./samples
workers                       : 6
scale_factor                  : 4
num_rrdn_blocks               : 23
nf                            : 64
gc                            : 32
b1                            : 0.9
b2                            : 0.999
weight_decay                  : 0.01
p_lr                          : 0.0002
p_decay_iter                  : [30, 60, 90]
p_perceptual_loss_factor      : 0
p_adversarial_loss_factor     : 0
p_content_loss_factor         : 1
g_lr                          : 0.0001
g_decay_iter                  : [25, 50, 80]
g_perceptual_loss_factor      : 1
g_adversarial_loss_factor     : 0.005
g_content_loss_factor         : 0.01
is_psnr_oriented              : True
load_previous_opt             : False

[Epoch 0/119] [Batch 1/563] [content loss 0.4679]
[Epoch 0/119] [Batch 282/563] [content loss 0.0799]
[Epoch 0/119] [Batch 563/563] [content loss 0.0761]

Epoch: 0 -> Gen loss: 0.0969 Con loss:: 0.0969
Validation Set: PSNR: 19.6505, SSIM:0.6475


[Epoch 1/119] [Batch 1/563] [content loss 0.0668]
[Epoch 1/119] [Batch 282/563] [content loss 0.0696]
[Epoch 1/119] [Batch 563/563] [content loss 0.0866]

Epoch: 1 -> Gen loss: 0.0806 Con loss:: 0.0806
Validation Set: PSNR: 19.658, SSIM:0.64425


[Epoch 2/119] [Batch 1/563] [content loss 0.0997]
[Epoch 2/119] [Batch 282/563] [content loss 0.0768]
[Epoch 2/119] [Batch 563/563] [content loss 0.0718]

Epoch: 2 -> Gen loss: 0.0797 Con loss:: 0.0797
Validation Set: PSNR: 19.85525, SSIM:0.647


[Epoch 3/119] [Batch 1/563] [content loss 0.0984]
[Epoch 3/119] [Batch 282/563] [content loss 0.0891]
[Epoch 3/119] [Batch 563/563] [content loss 0.1012]

Epoch: 3 -> Gen loss: 0.0795 Con loss:: 0.0795
Validation Set: PSNR: 19.779, SSIM:0.649


...

[Epoch 8/119] [Batch 1/563] [content loss 0.0624]
[Epoch 8/119] [Batch 282/563] [content loss 0.0812]
[Epoch 8/119] [Batch 563/563] [content loss 0.1138]

Epoch: 8 -> Gen loss: 0.0787 Con loss:: 0.0787
Validation Set: PSNR: 19.90875, SSIM:0.65625


[Epoch 9/119] [Batch 1/563] [content loss 0.0795]
[Epoch 9/119] [Batch 282/563] [content loss 0.0765]
[Epoch 9/119] [Batch 563/563] [content loss 0.0671]

Epoch: 9 -> Gen loss: 0.0785 Con loss:: 0.0785
Validation Set: PSNR: 19.897, SSIM:0.657


[Epoch 10/119] [Batch 1/563] [content loss 0.0646]
[Epoch 10/119] [Batch 282/563] [content loss 0.0695]
[Epoch 10/119] [Batch 563/563] [content loss 0.0965]

Epoch: 10 -> Gen loss: 0.0786 Con loss:: 0.0786
Validation Set: PSNR: 19.999000000000002, SSIM:0.657


[Epoch 11/119] [Batch 1/563] [content loss 0.0743]
[Epoch 11/119] [Batch 282/563] [content loss 0.0671]
[Epoch 11/119] [Batch 563/563] [content loss 0.0547]

Epoch: 11 -> Gen loss: 0.0785 Con loss:: 0.0785
Validation Set: PSNR: 19.74725, SSIM:0.6545


[Epoch 12/119] [Batch 1/563] [content loss 0.0805]
[Epoch 12/119] [Batch 282/563] [content loss 0.0986]


In [ ]:
pin = torch.cuda.is_available()

config["start_epoch"] = 108
config["num_epoch"] = 100
config["batch_size"] = 16
config["is_psnr_oriented"] = False
config["load_previous_opt"] = True


for key, value in config.items():
    print(f"{key:30}: {value}")

print("\n\n")
print(f"ESRGAN start")

torch.cuda.empty_cache()
gan_model = trainer.Trainer(
    config, esr_dataloader_train, esr_dataloader_val, device)
gan_model_metrics = gan_model.train()

In [ ]:
#@title Testing Images

import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import models
import importlib
importlib.reload(models)

checkpoint_number = 9

checkpoint = torch.load(rf"/content/checkpoint_{checkpoint_number}.tar")
Model = models.Generator(channels=3, nf=32, gc=32, num_res_blocks=11, scale=4)
Model.load_state_dict(checkpoint[rf"generator_dict_{checkpoint_number}"])
Model.to(device)
# print("Generator weights loaded.")


image_path = r"/content/images/lr/00000.png"


def test_image(image_path, model):

    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    cv2_imshow(image)
    image = image // 255.0
    image= image.astype(np.float32)

    image = np.moveaxis(image, (0, 1, 2), (1, 2, 0))
    image = torch.from_numpy(np.expand_dims(image, 0))
    with torch.no_grad():
        # with torch.cuda.amp.autocast():
        high_res = model(image.to(device))
    
    high_res = high_res.cpu().detach().permute(0, 2, 3, 1).numpy()
    cv2_imshow(high_res[0] * 255.0)
    print(high_res.shape)

test_image(image_path, Model)


# Model.eval()
# with torch.no_grad():
# with torch.cuda.amp.autocast(enabled=False):
    # out = Model(image)  



In [ ]:
# !rm -rf /content/samples
# !rm -rf /content/drive/MyDrive/Project-ESRGAN
# !rm -rf checkpoint*